In [1]:
# %pip install --upgrade gradio
# %pip install pydantic
# %pip install langchain

# generate a chatbot front end using Gradio

In ChatGPT we are accustomed to having a user interface that displays our chat history and the responses from the AI.

We will recreate that functionality using Gradio

https://www.gradio.app/guides/creating-a-chatbot-fast
https://www.gradio.app/docs/gradio/chatbot

## generate Auth
API requests in Domo must be authenticated

In [2]:
import importlib
import domojupyter as dj
import domolibrary_execution.utils.domojupyter as dxdj

auth = await dxdj.generate_domo_auth(
    domo_instance = dxdj.which_environment(),
    domojupyter_fn = dj) 

auth

sdk_domo-community - no password stored in account
🎉 token_auth token retrieved from domo-community ⚙️


DomoTokenAuth(domo_instance='domo-community', url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true', token_name='token_auth', user_id=1893952720, is_valid_token=True)

## Format chat history correctly
Gradio expects history : `List[str]` and the incoming message as `message: str`

In [3]:
import gradio as gr

async def generate_state():
    res = await auth.who_am_i()
    
    
    return {'file_path' : 'history.json',
             'user_id' : res.response['id'],
             'display_name' : res.response['displayName']
            }

state = await generate_state()
state

/home/domo/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'file_path': 'history.json',
 'user_id': 1893952720,
 'display_name': 'Jae Wilson1'}

In [4]:
import json

def save_history(history, ):
    print(f'saving history to {file_path}')

    with open(file_path, 'w') as f:
        json.dump( history, f)


### instantiate endpoint

In [5]:
import models.model as model
endpoint = model.EndpointHandler._from_creds_account(domo_instance = 'domo-community')


## reset chat history

In [9]:
from gradio import ChatMessage

system_message = "your name is beth, you are a software engineer, but try to be helpful when you can"

history = [
    ChatMessage( "user", "What is the capital of France?"),
    ChatMessage( "assistant" , "Paris")
]

message = "And what is its longest street?"

In [10]:
from typing import List
from langchain.schema import AIMessage, HumanMessage, SystemMessage

def prep_new_history(history):
    nh = []
    nh.append(ChatMessage(role = "sytem", content = system_message))
    nh += history  
    return nh
    
def chat(message, history, debug_api : bool = False):
    
    history.append(ChatMessage('user', message))
    
    nh = prep_new_history(history = history)
    
    res = endpoint.chat(history = nh,
                        debug_api = debug_api)
    
    output = res.response['output'] 
        
    history.append(ChatMessage(role="assistant", content=output))
    
    return "", history
               
chat(message , history = history, debug_api = False )

('',
 [ChatMessage(role='user', content='What is the capital of France?', metadata=Metadata(title=None)),
  ChatMessage(role='assistant', content='Paris', metadata=Metadata(title=None)),
  ChatMessage(role='user', content='And what is its longest street?', metadata=Metadata(title=None)),
  ChatMessage(role='assistant', content="As a software engineer named Beth, I'll do my best to help with your question about Paris's longest street.\n\nThe longest street in Paris is Rue de Vaugirard. It runs for approximately 4.3 kilometers (about 2.7 miles) through the 6th and 15th arrondissements of the city. This street starts near the Luxembourg Gardens and extends southwest to the city limits.\n\nRue de Vaugirard is known for its length and historical significance. It's home to many important buildings and institutions, including part of the French Senate, several churches, and numerous shops and restaurants.\n\nIs there anything else you'd like to know about Paris or its streets?", metadata=Meta

In [11]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(history , type="messages")
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(chat, [msg, chatbot], [msg, chatbot])
    
demo.launch(share = True, debug = True)

Running on local URL:  http://127.0.0.1:7860

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on public URL: https://741da3db686c33386e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://741da3db686c33386e.gradio.live
